# Simulador de Lanzamiento de Dados

In [1]:
import random as rd
import matplotlib.pyplot as plt
import numpy as np

import sys
sys.path.append("../src")

from simulador import Simulador


In [2]:
caras_dado=6
nLanzamientos=1000
nDados=3

In [3]:
Simulador.dados(caras_dado,nDados)
lanzamintos=Simulador.lanzamientos(nLanzamientos)
print(lanzamintos)

[[1 6 1]
 [6 2 1]
 [3 6 1]
 ...
 [2 4 2]
 [1 4 3]
 [4 4 4]]


In [4]:
def dato(ndado,caras,resultados):
    frecuencias={}
    for cara,resultado in zip(caras,resultados):
        frecuencias[int(cara)]=int(resultado)
    return {f"dado_{ndado}":frecuencias}

In [5]:
def frecuencia_absoluta(nDados,lanzamintos):
    resultados={}
    for i in range(nDados):
        cara_dado, frecuencias_cara= np.unique(lanzamintos[:, i], return_counts=True)
        resultados[f"dado_{i+1}"]=frecuencias_cara
    return resultados

In [6]:
# frecuencia absoluta
frecuencia=frecuencia_absoluta(nDados,lanzamintos)
frecuencia

{'dado_1': array([163, 175, 164, 159, 173, 166]),
 'dado_2': array([167, 180, 170, 170, 163, 150]),
 'dado_3': array([182, 159, 178, 141, 168, 172])}

In [7]:
# frecuencia relativa
def frecuencia_relativa_dado(nLanzamientos,frecuencia):
    resultados={}
    for dado, frecuencias in frecuencia.items():
        resultados[dado]=frecuencias/nLanzamientos
    return resultados

frecuencia_relativa_dado(nLanzamientos,frecuencia)

{'dado_1': array([0.163, 0.175, 0.164, 0.159, 0.173, 0.166]),
 'dado_2': array([0.167, 0.18 , 0.17 , 0.17 , 0.163, 0.15 ]),
 'dado_3': array([0.182, 0.159, 0.178, 0.141, 0.168, 0.172])}

In [8]:
def imprimir_resultados(probabilidad):
    for suma, p in probabilidad.items():
        print(f"Suma {suma}: {p:.4f}")

In [9]:
import itertools
from collections import Counter
# frecuencia relativa teorica
def probabilidad_teorica(caras_dado=6, nDados=1):
    combinaciones = itertools.product(range(1, caras_dado + 1), repeat=nDados)
    sumas = (sum(comb) for comb in combinaciones)
    conteo = Counter(sumas)
    total_combinaciones = caras_dado ** nDados
    probabilidad = {suma: conteo[suma] / total_combinaciones for suma in sorted(conteo)}
    return probabilidad

imprimir_resultados(probabilidad_teorica(caras_dado, nDados))

Suma 3: 0.0046
Suma 4: 0.0139
Suma 5: 0.0278
Suma 6: 0.0463
Suma 7: 0.0694
Suma 8: 0.0972
Suma 9: 0.1157
Suma 10: 0.1250
Suma 11: 0.1250
Suma 12: 0.1157
Suma 13: 0.0972
Suma 14: 0.0694
Suma 15: 0.0463
Suma 16: 0.0278
Suma 17: 0.0139
Suma 18: 0.0046


In [10]:
# frecuen relativa real
def frecuencia_relativa(lanzamintos):
    sumas = (sum(comb) for comb in lanzamintos)
    conteo = Counter(sumas)
    total_combinaciones = len(lanzamintos)
    probabilidad = {suma: conteo[suma] / total_combinaciones for suma in sorted(conteo)}
    return probabilidad

imprimir_resultados(frecuencia_relativa(lanzamintos))

Suma 3: 0.0030
Suma 4: 0.0240
Suma 5: 0.0340
Suma 6: 0.0440
Suma 7: 0.0620
Suma 8: 0.1020
Suma 9: 0.1090
Suma 10: 0.1250
Suma 11: 0.1300
Suma 12: 0.1150
Suma 13: 0.1020
Suma 14: 0.0710
Suma 15: 0.0400
Suma 16: 0.0160
Suma 17: 0.0160
Suma 18: 0.0070


In [11]:
import matplotlib.pyplot as plt

# frecuancia teorica

In [12]:
prob_teorica=probabilidad_teorica(caras_dado, nDados)

## prueba con 10 lanzamiento

In [13]:
Simulador.dados(caras_dado,nDados)
lanzamintos=Simulador.lanzamientos(10)
fr10=frecuencia_relativa(lanzamintos)
imprimir_resultados(fr10)

Suma 7: 0.1000
Suma 8: 0.1000
Suma 10: 0.1000
Suma 12: 0.3000
Suma 13: 0.2000
Suma 14: 0.1000
Suma 16: 0.1000


In [14]:
#! falta las graficas de barra para comparar

## prueba con 100 lanzamiento

In [15]:
Simulador.dados(caras_dado,nDados)
lanzamintos=Simulador.lanzamientos(100)
fr100=frecuencia_relativa(lanzamintos)
imprimir_resultados(fr100)

Suma 4: 0.0100
Suma 5: 0.0300
Suma 6: 0.0600
Suma 7: 0.0600
Suma 8: 0.1500
Suma 9: 0.1000
Suma 10: 0.1000
Suma 11: 0.1000
Suma 12: 0.1200
Suma 13: 0.1100
Suma 14: 0.0700
Suma 15: 0.0500
Suma 16: 0.0300
Suma 17: 0.0100


## prueba con 1000000 lanzamiento

In [16]:
Simulador.dados(caras_dado,nDados)
lanzamintos=Simulador.lanzamientos(1000000)
fr1000000=frecuencia_relativa(lanzamintos)
imprimir_resultados(fr1000000)

Suma 3: 0.0047
Suma 4: 0.0138
Suma 5: 0.0276
Suma 6: 0.0468
Suma 7: 0.0699
Suma 8: 0.0970
Suma 9: 0.1156
Suma 10: 0.1246
Suma 11: 0.1251
Suma 12: 0.1155
Suma 13: 0.0978
Suma 14: 0.0696
Suma 15: 0.0462
Suma 16: 0.0274
Suma 17: 0.0138
Suma 18: 0.0045
